<a href="https://colab.research.google.com/github/rainbow2125514/CS114.K21/blob/master/Sacarsm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Các thành viên thực hiện:
1.   Nguyễn Võ Hùng Vỹ : 18521683
2.   Nguyễn Thị Hà : 1852691
3.   Nguyễn Hải Ngọc : 18520321

## **Đề bài:Sacarsm detection**


Bài toán phát hiện tin châm biếm - News headline sarcasm detection sử dụng dataset https://www.kaggle.com/rmisra/news-headlines-dataset-for-sarcasm-detection là bài toán con trong lĩnh vực Natural language processing và khá gần một bài toán hot hiện nay là fake news detection. 
Đây là một bài toán mở (open problem) vẫn chưa có phương pháp nào có độ chính xác đủ cao để ứng dụng  trong thực tiễn. Dataset được chọn là dataset chuẩn mực về mặt ngôn ngữ và cách xây dựng dataset cũng khá đơn giản, sinh viên có thể tự thực hiện được nếu cần bổ sung thêm dataset cho mình.

# bước 1 : lấy dữ liệu từ kaggle

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vynguyenvohung","key":"5c183cde27c116862c145e1f7b184b9e"}'}

In [3]:
pip install kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets list -s sarcasm

ref                                                      title                                               size  lastUpdated          downloadCount  
-------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  
danofer/sarcasm                                          Sarcasm on Reddit                                  216MB  2018-05-27 08:19:04           4586  
rmisra/news-headlines-dataset-for-sarcasm-detection      News Headlines Dataset For Sarcasm Detection         3MB  2019-07-03 23:52:57          15607  
rmisra/news-category-dataset                             News Category Dataset                               25MB  2018-12-02 04:09:45          13215  
sherinclaudia/sarcastic-comments-on-reddit               Sarcastic Comments - REDDIT                        106MB  2019-01-30 14:41:53           1931  
rmisra/clothing-fit-dataset-for-size-recommendation      Clothing Fit Dataset for Size R

In [6]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection


  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [7]:
!mkdir -p ~/.kaggle/datasets
!unzip news-headlines-dataset-for-sarcasm-detection.zip -d ~/.kaggle/datasets

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: /root/.kaggle/datasets/Sarcasm_Headlines_Dataset.json  
  inflating: /root/.kaggle/datasets/Sarcasm_Headlines_Dataset_v2.json  


In [8]:
cd ~/.kaggle/datasets/

/root/.kaggle/datasets


# bước 2 : xử lý dữ liệu

In [9]:
import pandas as pd

datasets=pd.read_json('Sarcasm_Headlines_Dataset_v2.json',lines=True)

In [10]:
import re
import string

#tạo ra 1 funtion để loại bỏ các ký tự đặc biệt như *&^%...., chuyển thành viết thường và bỏ khoảng cách 2 bên
def xuli(headline):
  headline=''.join(ch for ch in headline if ch in (string.ascii_letters+string.digits+' '))
  headline=headline.lower()
  headline=headline.strip()
  return headline

In [11]:
#tạo ra list corpus để chứa dữ liệu sau khi xử lý
corpus=[]
for a in datasets['headline']:
  corpus.append(xuli(a))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

fe=TfidfVectorizer(stop_words=("a","the","an","of","with",'english'))
datas=fe.fit_transform(corpus)
targets=datasets["is_sarcastic"].values

In [13]:
from sklearn.model_selection import train_test_split

#chia thành 2 tập : tập train và tập test
x_train,x_test,y_train,y_test=train_test_split(datas,targets,train_size=0.6)

In [14]:
from sklearn.linear_model import LogisticRegression

#dùng model LogisticRegression và train
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

# Bước 3 : đánh giá dữ liệu

In [16]:
from sklearn.metrics import accuracy_score

#dự đoán và đánh giá dựa vào 
y_pred=model.predict(x_test)
print('accuracy score in model LogisticRegression :{:.2f} %'.format(accuracy_score(y_pred,y_test)*100))

accuracy score in model LogisticRegression :81.91 %


In [19]:
from sklearn.metrics import confusion_matrix

print("confusion matrix:")
print(confusion_matrix(y_pred,y_test))


confusion matrix:
[[4955 1038]
 [1033 4422]]


# Bước 4 : dự đoán

Dữ liệu được lấy thông qua code trong : https://github.com/rainbow2125514/CS114.K21/blob/master/crawHeadline.ipynb

sau đó được nén trong file data.csv

In [20]:
!git clone https://github.com/rainbow2125514/datasets

fatal: destination path 'datasets' already exists and is not an empty directory.


In [21]:
cd ./datasets

/root/.kaggle/datasets/datasets


In [22]:
import pandas as pd

dataset_pred=pd.read_csv("data.csv")

In [23]:
dataset_pred["headline"]

0       Governors Pause Some Reopenings As Coronavirus...
1                         House Passes Police Reform Bill
2       The Supreme Court Rulings On LGBTQ Rights And ...
3       Maxine Waters Rips Trump For Focusing More On ...
4       Yes, The Government Mailed Stimulus Checks To ...
                              ...                        
2209    Al Franken: ‘I’m Deeply Sorry For My Hilarious...
2210    Trump Promises U.S. Will Continue To Recognize...
2211    Ryan Zinke Comes Out In Support Of Controversi...
2212    New RNC Ad Endorses Roy Moore: ‘He’s A Scumbag...
2213    Flynn Pleads Guilty To Lying To FBI, But, Wors...
Name: headline, Length: 2214, dtype: object

In [36]:
b=[]
for a in dataset_pred["headline"]:
  b.append(xuli(a))
x2_pred=fe.transform(b)
y2_pred=model.predict(x2_pred)
pd_data_pred=pd.DataFrame()
pd_data_pred["headline"]=b
pd_data_pred["is_sarcastic predict"]=y2_pred
pd_data_pred

,headline,is_sarcastic predict
0,governors pause some reopenings as coronavirus...,0
1,house passes police reform bill,0
2,the supreme court rulings on lgbtq rights and ...,0
3,maxine waters rips trump for focusing more on ...,0
4,yes the government mailed stimulus checks to d...,0
...,...,...
2209,al franken im deeply sorry for my hilarious ac...,0
2210,trump promises us will continue to recognize p...,0
2211,ryan zinke comes out in support of controversi...,1
2212,new rnc ad endorses roy moore hes a scumbag bu...,0
